## Deliverable 2. Create a Customer Travel Destinations Map.

In [21]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from VacayConfig import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [6]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Thompson,CA,55.7435,-97.8558,14.16,73,100,13.80,overcast clouds
1,1,Albany,US,42.6001,-73.9662,31.37,74,32,1.99,scattered clouds
2,2,Ushuaia,AR,-54.8000,-68.3000,49.66,57,20,11.50,few clouds
3,3,Jamestown,US,42.0970,-79.2353,27.72,79,100,10.36,overcast clouds
4,4,Busselton,AU,-33.6500,115.3333,68.04,57,0,6.31,clear sky


In [8]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp=float(input("What is your minimum acceptable temperature for your trip?"))
max_temp=float(input("What is your maximum acceptable temperature for your trip?"))

What is your minimum acceptable temperature for your trip?60
What is your maximum acceptable temperature for your trip?90


In [9]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
city_temp_df=city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & 
                              (city_data_df["Max Temp"] <= max_temp)]
city_temp_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Busselton,AU,-33.6500,115.3333,68.04,57,0,6.31,clear sky
8,8,Rikitea,PF,-23.1203,-134.9692,78.96,71,81,13.94,light rain
9,9,Sinnamary,GF,5.3833,-52.9500,76.80,85,29,5.99,scattered clouds
11,11,Cape Town,ZA,-33.9258,18.4232,65.08,82,0,4.61,clear sky
12,12,Hobart,AU,-42.8794,147.3294,78.26,53,20,9.22,few clouds


In [10]:
# 4a. Determine if there are any empty rows.
city_temp_df.isnull().sum()

City_ID                0
City                   0
Country                4
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [12]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
city_temp_clean_df=city_temp_df.dropna()

city_temp_clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [13]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = city_temp_clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Busselton,AU,68.04,clear sky,-33.6500,115.3333,
8,Rikitea,PF,78.96,light rain,-23.1203,-134.9692,
9,Sinnamary,GF,76.80,scattered clouds,5.3833,-52.9500,
11,Cape Town,ZA,65.08,clear sky,-33.9258,18.4232,
12,Hobart,AU,78.26,few clouds,-42.8794,147.3294,
13,Karangampel,ID,80.35,overcast clouds,-6.4622,108.4519,
14,Butaritari,KI,82.27,light rain,3.0707,172.7902,
17,Te Anau,NZ,68.25,overcast clouds,-45.4167,167.7167,
18,Hilo,US,76.24,broken clouds,19.7297,-155.0900,
19,Lisala,CD,69.49,broken clouds,2.1500,21.5167,


In [15]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("hotel not found..skipping")    
        

hotel not found..skipping
hotel not found..skipping
hotel not found..skipping
hotel not found..skipping
hotel not found..skipping
hotel not found..skipping
hotel not found..skipping
hotel not found..skipping
hotel not found..skipping
hotel not found..skipping
hotel not found..skipping
hotel not found..skipping
hotel not found..skipping
hotel not found..skipping
hotel not found..skipping
hotel not found..skipping
hotel not found..skipping
hotel not found..skipping
hotel not found..skipping
hotel not found..skipping
hotel not found..skipping
hotel not found..skipping
hotel not found..skipping


In [24]:
# 7. Drop the rows where there is no Hotel Name.
clean=hotel_df.replace(r'^\s*$', np.nan, regex=True)
clean_hotel_df=hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Busselton,AU,68.04,clear sky,-33.6500,115.3333,Observatory Guest House
8,Rikitea,PF,78.96,light rain,-23.1203,-134.9692,People ThankYou
9,Sinnamary,GF,76.80,scattered clouds,5.3833,-52.9500,ALETHEIA Appartement Sinnamary
11,Cape Town,ZA,65.08,clear sky,-33.9258,18.4232,Southern Sun Waterfront Cape Town
12,Hobart,AU,78.26,few clouds,-42.8794,147.3294,Mantra on Collins Hobart


In [26]:
# 8a. Create the output File (CSV)
output_data_file1="WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file1, index_label="City_ID")

In [27]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [29]:
# 11a. Add a marker layer for each city to the map. 
mark_layer=gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
figure=gmaps.figure(center=(30,31), zoom_level=1.7)
figure.add_layer(mark_layer)
figure

Figure(layout=FigureLayout(height='420px'))